We are going to implement a code for Linear Quadratic Control problem 

Loss function = $\dfrac1M\sum_{t=0}^{N-1}((aX_t^2+bX_t+Au_t^2+Bu_t)\delta_t+\alpha X_T^2+\beta X_T)$

where $\delta_t=\frac{N}{T}$

State Process $X_{t+1}=X_t+(cX_t+du_t)\delta_t+\sigma\delta_t\epsilon_t$

The HJB equation is given by

$0=\partial_t V(t,x)+\frac{\sigma^2}{2}\partial_x^2V(t,x) +cx\partial_xV(t,x)+ax^2+bx+\inf_u\Big[Au^2+\Big(B+d\partial_xV(t,x)\Big)u\Big]$

$V(T,x)=\alpha x^2+\beta x$.

The optimal solution solution obtained as
$u^*=-\frac{B+d\partial_xV(t,x)}{2A}$
depends on the value function $V(t,x)$, which is the solution of the following PDE
$0=\partial_t V(t,x)+\frac{\sigma^2}{2}\partial_x^2V(t,x) +cx\partial_xV(t,x)+ax^2+bx-\frac{\big(B+d\partial_xV(t,x)\big)^2}{4A}$

$V(T,x)=\alpha x^2+\beta x$.

We assume that the value function is quadratic in $x$,

$V(t,x)=f(t)x^2+h(t)x+k(t)$. 

By substituting $V$ in the HJB PDE we get the following system of Riccati ODE.

$0=f'+a+2cf-\frac{d^2}{A}f^2;  f(T)=\alpha$

$0=h'+b-\frac{(B+dh)d}{A}f;  h(T)=\beta$

$0=k'+\sigma^2 f-\frac{1}{4A}(B+dh)^2;  k(T)=0$

By solving this system of first order ODE we can get $f,h,t$ in closed form and we obtain the optimal solution as

$u^*=-\frac{B+d(2xf(t)+h(t))}{2A}$

In [21]:
import numpy as np
import time
from scipy import misc
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
import pandas as pd
from scipy.optimize import fsolve
from scipy.integrate import solve_ivp

In [22]:
M=10000
ite=100
T=1
delta = torch.tensor(T/ite)
a=1
b=1
c=1
d=1
A=1
B=1
sigma=0.01
alpa=1
beta=1
neuron_model_psi=50
dim_n=2
print(delta)

tensor(0.0100)


In [23]:
# model= torch.nn.Sequential(
#     torch.nn.Linear(dim_n, neuron_model_psi),
#     torch.nn.ReLU(),
# #     torch.nn.Linear(neuron_model_psi, neuron_model_psi),
# #     torch.nn.ReLU(),
# #     torch.nn.Linear(neuron_model_psi, neuron_model_psi),
# #     torch.nn.ReLU(),
# #     torch.nn.Linear(neuron_model_psi, neuron_model_psi),
# #     torch.nn.ReLU(),
#     torch.nn.Linear(neuron_model_psi,1)
# )

In [24]:
model= torch.load('model_N100_LQC.pth')


In [25]:
t=torch.zeros([M,1])
up_lim=100
#z=np.zeros([M,ite+1])+np.random.rand(M,1)
z=torch.randn(size=(M, ite+1))
x_train=torch.FloatTensor(M,1).uniform_(0,up_lim)   #remaining balance R_t   #To get a positive solution R_t has to be greater than D_t
data=torch.cat((t,x_train),dim=1)
# print(data.dtype)
# print(z.dtype)
# print(z[:,2].shape)
#print(x)

In [26]:
def update(x3,psi,z,i):
    t=(x3[:,0]+(T/ite))
    #print(x3[:,1].shape,z[:,i].shape,psi.shape)
    x=x3[:,1]+(c*x3[:,1]+d*psi)*delta+sigma*torch.sqrt(delta)*z[:,i]
    up=torch.cat((t.unsqueeze(1),x.unsqueeze(1)),dim=1)
#     print('up shape=', up.shape)
    return up

def loss_func(x2,psi):
    loss=a*torch.pow(x2[:,1],2)+b*x2[:,1]+A*torch.pow(psi,2)+B*psi
    return loss

def loss_func_ter(x4):
    loss=alpa*torch.pow(x4[:,1],2)+beta*x4[:,1]
    return loss

def unit(x1,model,i):
    psi=model(x1).squeeze(1)
    los=loss_func(x1,psi)
    upd=update(x1,psi,z,i)
    #print('unit print=',psi.shape,los.shape,upd.shape)
    return psi,los,upd

def loss_func_total(u,model,epoch):
    loss=torch.zeros(M,ite)
    psi=torch.zeros(M,ite)
    state_var=[]
    if (epoch==max_epoch-1):
        state_var.append(u.detach().numpy())   
    for i in range(ite+1):
        if(i<ite):
            psi_run,loss_run,u_run=unit(u,model,i)
            #print('los func=',psi_run.shape,loss_run.shape)
            loss[:,i]=loss_run
            #print(loss)
            psi[:,i]=psi_run
            #print(psi)
            u=u_run
            if (epoch==max_epoch-1):
                state_var.append(u.detach().numpy())             
            #print(u)
        else:
            loss_ter=loss_func_ter(u)
          #print('ter',loss_ter.shape)
        #print(torch.sum(loss,dim=1))
    loss=torch.sum(loss,dim=1)*delta+loss_ter
    #print(loss.shape)
    return torch.mean(loss),state_var


In [27]:
# start=time.time()
# lr1 = 8e-3
# max_epoch = 5000
# cost_ar=[]
# optimizer = optim.Adam(model.parameters(), lr1)

# #print(psi.shape)
# for epoch in range(max_epoch):
#   optimizer.zero_grad()
#   cost,state_var_train=loss_func_total(data,model,epoch)
#   cost.backward()
#   optimizer.step()
#   cost_ar.append(cost)
#   #print(loss.item())
#   if (epoch % 100==0):
#     print("At epoch {} the mean cost is {}.".format(epoch,cost.detach()))
# end=time.time()
# total_time=end-start
# print('Training time=',total_time)


In [29]:
# Define the system of ODEs
def system_of_odes(t, y):
    dydt = [
        a+2*c*y[0]-(np.power(d,2)*np.power(y[0],2))/A,                 # df/dt = a+2cf-d^2*f^2/A; f(0)=alpa
        b-(B+d*y[1])*d*y[0]/A,                       # dh/dt = b-(B+dh)fd/A; h(0)=beta
        np.power(sigma,2)*y[0]-(B+d*y[1])**2/(4*A)        # dk/dt = sigma^2*f-(B+dh)^2/4A; k(0)=0
    ]
    return dydt

# Initial conditions
initial_conditions = [alpa, beta, 0]  # Initial values for y1, y2, and y3

# Time span
t_span = (0, T)  # Start and end time

# Solve the system of ODEs
sol = solve_ivp(system_of_odes,t_span, initial_conditions, t_eval=np.linspace(0, T, ite))

f=np.flip(sol.y[0])
h=np.flip(sol.y[1])
k=np.flip(sol.y[2])
# Plot the solution
plt.plot(sol.t, f, label='f')
plt.plot(sol.t, h, label='h')
plt.plot(sol.t, k, label='k')
plt.xlabel('Time')
plt.ylabel('Solution')
plt.title('Solution of the system of ODEs')

plt.legend()
plt.grid(True)
plt.show()


In [31]:
def optimal_soln(x,f,h):
    u=-(B+d*(2*x*f+h))/(2*A)
    return u

def value_func(x,f,h,k):
    V=f*np.power(x,2)+h*x+k
    return V

def total_cost(x,u):
    cost_run=0.0
    for i in range(ite+1):
        if(i<ite):
            cost_run=cost_run+a*np.power(x[i],2)+b*x[i]+A*np.power(u[i],2)+B*u[i]
        else:
            cost_ter=alpa*np.power(x[i],2)+beta*x[i]
    return cost_run*delta.numpy()+cost_ter    

# Neural Network Prediction

In [32]:
M1=2
x_ar=[]
z1=torch.randn(size=(M1,ite))
t_test=torch.zeros([M1,1])
x1=torch.linspace(0.3*up_lim,0.5*up_lim, steps=M1).unsqueeze(1)
x_test=torch.cat((t_test,x1),dim=1)
pred_soln=torch.zeros(M1,ite)
for i in range(ite):
    x_ar.append(x_test[:,1].numpy())
    soln_pred=model(x_test).squeeze(1).detach()
    pred_soln[:,i]=soln_pred
    x_test=update(x_test,soln_pred,z1,i)
x_ar.append(x_test[:,1].numpy())
state_var=np.squeeze(np.array(x_ar))
state_var=state_var.transpose()
print(state_var.shape)

(2, 101)


In [33]:
t_eval=np.linspace(0,T,ite)
u=optimal_soln(state_var[:, :-1],f,h)
V=value_func(state_var[:, :-1],f,h,k)
pred=pred_soln.numpy()
for i in range(M1):
    print('state variable',state_var[i])
    print('\n True solution',u[i])
    true_cost=total_cost(state_var[i],u[i])
    print('\n True cost=',true_cost)
    print('\n Predicted solution',pred[i])
    pred_cost=total_cost(state_var[i],pred[i])
    print('\n Predicted cost=',pred_cost)
    print('Percent Error in cost=',np.abs(100*(np.abs(true_cost)-np.abs(pred_cost))/true_cost),'%\n')

state variable [30.        29.61633   29.23922   28.871119  28.50726   28.149557
 27.798613  27.452639  27.113577  26.778755  26.449669  26.1274
 25.810894  25.496933  25.190178  24.888744  24.59139   24.299303
 24.010561  23.729345  23.451517  23.177889  22.909266  22.647219
 22.388756  22.133944  21.881716  21.633806  21.391266  21.152245
 20.91752   20.68486   20.457964  20.23566   20.018408  19.802923
 19.594303  19.388027  19.18876   18.991453  18.798862  18.610199
 18.424961  18.241941  18.063915  17.889606  17.71763   17.548956
 17.383596  17.22283   17.066399  16.91328   16.762201  16.613934
 16.468824  16.328022  16.191992  16.055752  15.922619  15.79455
 15.669603  15.54967   15.431266  15.314354  15.201418  15.091229
 14.98379   14.881279  14.782536  14.686631  14.59435   14.5044775
 14.418172  14.335942  14.257412  14.180966  14.108757  14.038474
 13.972255  13.907515  13.843359  13.7842455 13.72545   13.670226
 13.618443  13.568296  13.52183   13.476408  13.43425   13.3940

# Optimal Solution at different time steps

In [ ]:
import matplotlib.pyplot as plt
# Create subplots
fig, axs = plt.subplots(ite, 1, figsize=(9, 40))
ig, axs = plt.subplots(ite, 1)
state_var=state_var[:, :-1]
for i, ax in enumerate(axs):
    ax.scatter(state_var[:,i],pred[:,i],color='r',label='X vs Pred', marker='o')
    ax.scatter(state_var[:,i],u[:,i], color='blue', label='X vs True', marker='*')
    ax.set_xlabel('State Variable X')
    ax.set_ylabel('Optimal Soln')
    ax.set_title(f'Optimal Solution at Time Step {i}')
    ax.legend()
# Adjust layout
plt.tight_layout()
plt.show()

# Optimal solution for different sample path

In [13]:
matplotlib.use('Qt5Agg')
#fig, axs = plt.subplots(M1,1, figsize=(6, 25))
fig, axs = plt.subplots(M1,1)

# Plot x vs y and x vs z on each subplot
for i, ax in enumerate(axs):
    ax.plot(t_eval,pred[i], color='b', label='Predicted')
    ax.plot(t_eval,u[i], color='r', label='Closed Form')
    ax.set_xlabel('Time')
    ax.set_ylabel('Optimal Solution')
    ax.set_title(f'Optimal solution for the sample path of X_0 = {state_var[i,0]:.2f}')
    ax.legend()

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()

In [14]:
#torch.save(model, 'model_N10_LQC.pth')


# Surface plot for the optimal solution (closed form)

In [15]:

from mpl_toolkits.mplot3d import Axes3D


x_grid, y_grid = np.meshgrid(state_var[1], t_eval)     # create a grid of x and y values
z= optimal_soln(x_grid,f,h)  # z values, here we just take sin(r), r = sqrt(x^2 + y^2)

# Create a figure and a 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(x_grid, y_grid, z, cmap='viridis')  # 'viridis' is just a colormap, you can change it

# Add a color bar which maps values to colors
fig.colorbar(surf)

# Set labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('T Label')
ax.set_zlabel('U Label')
ax.set_title('3D Surface Plot for Optimal solution')

# Show plot
plt.show()


# Surface plot for the Value function

In [16]:

V= value_func(x_grid,f,h,k)  # z values, here we just take sin(r), r = sqrt(x^2 + y^2)

# Create a figure and a 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(x_grid, y_grid, V, cmap='viridis')  # 'viridis' is just a colormap, you can change it

# Add a color bar which maps values to colors
fig.colorbar(surf)

# Set labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('T Label')
ax.set_zlabel('U Label')
ax.set_title('3D Surface Plot for Value function')

# Show plot
plt.show()

In [17]:
# state_var_train=np.squeeze(np.array(state_var_train))

# state_var_dic = {'state variable':state_var_train}

# np.savez('state_var_dic_N10_new.npz',**state_var_dic)

In [18]:
# data = np.load('state_var_dic_N10.npz')

# # Accessing the arrays
# array1 = data['state variable']
# data.close()
# print(array1)

In [19]:
# import json
# cost_numpy_list = [tensor.detach().item() for tensor in cost_ar]
# #state_var_train_list=[tensor.detach().tolist() for tensor in state_var_train]

# my_dic = {'Num_sample':M,'Time_step':ite,'T':T,'a':a,'b':b,'c':c,'d':d,'A':A,'B':B,
#           'alpa':alpa,'beta':beta,'Num_Neoron':neuron_model_psi,'Training_time':total_time,'epoch':max_epoch,
#            'accuracy':10,'Training loss':cost_numpy_list}

# file_path=  'N10_lqc_dic.json'
# with open(file_path,'w') as file:
#     json.dump(my_dic,file)
